In [19]:
import pandas as pd
# import os
# import shutil

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

df = pd.read_csv(r'sarcasm_subset.csv')
frames = [df]
full = pd.concat(frames)
full

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,Bushie the other Bushie the next Bushie the af...,selicos,politics,606,-1,-1,2016-10,2016-10-13 02:39:41,"Don't you mean, Bushie the Younger?"
1,1,Congrats on getting such damage before they we...,SenGenketsu,Robocraft,1,-1,-1,2016-10,2016-10-27 16:51:51,"So, Flak will be a thing again? Than i have to..."
2,0,Keeper of the Grove too,pucykoks,hearthstone,14,14,0,2016-09,2016-09-13 21:33:07,Especially when you compare it to dalaran Mage...
3,1,'This would be like my form of alcoholism' cle...,redditzendave,politics,7,-1,-1,2016-10,2016-10-21 19:16:31,See It: Trump hires teen because she's beautif...
4,1,Great input,Karieo,wow,2,2,0,2016-09,2016-09-19 20:54:15,you swap to druid.
...,...,...,...,...,...,...,...,...,...,...
99995,0,"Alright lmao, a bit *too* gay for my tastes",ScootaliciousScooter,teenagers,2,-1,-1,2016-12,2016-12-30 04:05:43,"This is just getting too far, sort of like you..."
99996,1,Rip this is the farthest I've ever gotten,EmceeSexy,teenagers,1,-1,-1,2016-12,2016-12-05 02:18:19,ok
99997,1,Was expecting a tutorial on how to doxx your d...,Orrangejuiced,h1z1,3,-1,-1,2016-12,2016-12-07 04:48:41,Ninja tutorial
99998,1,He just wants more time to prepare for playing...,bamachine,CFB,2,-1,-1,2016-11,2016-11-23 03:55:23,Coaches Poll: Most feel Jimbo Fisher leaves fo...


In [20]:
text_data = list(df["comment"])[:5000]
class_data = df["label"].to_numpy()[:5000]
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
        
# for i in text_data:
#     print(type(i))

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the pre-trained BERT model and tokenizer
def BertTestAccF1(modelpath, text):
    model = torch.load(modelpath)
    model.to(device)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Load the training data
    # train_texts = ["Example sentence 1", "Example sentence 2", ...]
    # train_labels = [0, 1, ...]

    # Tokenize the text data
    train_encodings = tokenizer(list(text_data), padding=True, truncation=True, max_length=128)
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(list(class_data))
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    f1_metric = BinaryF1Score().to(device)

    acc = []
    f1 = []
    for idx, batch in enumerate(train_loader):
        outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        outputs = torch.argmax(outputs, axis = 1)
        acc.append((outputs.cpu() == batch[2]).float().sum().numpy())
        f1.append(f1_metric(outputs, batch[2].to(device)).cpu().detach().numpy())

    print(f"Accuracy of {text}: ", (sum(acc) / 5000) * 100, "%")
    print(f"F1 Score of {text}: ", (sum(f1) / len(f1)) * 100, "%")
    print()
    
    
BertTestAccF1("models/BERT.pth", "BERT")
BertTestAccF1("models/BERTTransfer.pth", "BERT Transfer IMDB")
BertTestAccF1("models/BERTTransferIrony.pth", "BERT Transfer Irony")
BertTestAccF1("models/BERTTransferHumor.pth", "BERT Transfer Humor")

Accuracy of BERT:  71.7 %
F1 Score of BERT:  83.25725877360934 %

Accuracy of BERT Transfer IMDB:  74.83999999999999 %
F1 Score of BERT Transfer IMDB:  85.42702179046194 %

Accuracy of BERT Transfer Irony:  67.24 %
F1 Score of BERT Transfer Irony:  80.15246133136141 %

Accuracy of BERT Transfer Humor:  73.18 %
F1 Score of BERT Transfer Humor:  84.21637229858689 %



In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from torchmetrics.classification import BinaryF1Score, BinaryPrecisionRecallCurve


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the pre-trained BERT model and tokenizer
def RoBertaTestAccF1(modelpath, text):
    model = torch.load(modelpath)
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')

    # Load the training data
    # train_texts = ["Example sentence 1", "Example sentence 2", ...]
    # train_labels = [0, 1, ...]

    # Tokenize the text data
    train_encodings = tokenizer(list(text_data), padding=True, truncation=True, max_length=128)
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(list(class_data))
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    f1_metric = BinaryF1Score().to(device)

    acc = []
    f1 = []
    for idx, batch in enumerate(train_loader):
        outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        outputs = torch.argmax(outputs, axis = 1)
        acc.append((outputs.cpu() == batch[2]).float().sum().numpy())
        f1.append(f1_metric(outputs, batch[2].to(device)).cpu().detach().numpy())

    print(f"Accuracy of {text}: ", (sum(acc) / 5000) * 100, "%")
    print(f"F1 Score of {text}: ", (sum(f1) / len(f1)) * 100, "%")
    print()
    
    
RoBertaTestAccF1("models/RoBERTa.pth", "RoBERTa")
RoBertaTestAccF1("models/RoBERTaTransfer.pth", "RoBERTa Transfer IMDB")
RoBertaTestAccF1("models/RoBERTaTransferIrony.pth", "RoBERTa Transfer Irony")
RoBertaTestAccF1("models/RoBERTaTransferHumor.pth", "RoBERTa Transfer Humor")

Accuracy of RoBERTa:  66.64 %
F1 Score of RoBERTa:  79.76926238673508 %

Accuracy of RoBERTa Transfer IMDB:  72.88 %
F1 Score of RoBERTa Transfer IMDB:  84.0859683455935 %

Accuracy of RoBERTa Transfer Irony:  64.64 %
F1 Score of RoBERTa Transfer Irony:  78.24215167646955 %

Accuracy of RoBERTa Transfer Humor:  71.86 %
F1 Score of RoBERTa Transfer Humor:  83.37917430385663 %



In [6]:
from torchmetrics.classification import BinaryF1Score, BinaryPrecisionRecallCurve
from transformers import AutoTokenizer, AutoModel, BertModel, BertTokenizer 

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the pre-trained BERT model and tokenizer
model_1 = torch.load('models/BERT.pth')
model_2 = torch.load('models/RoBERTa.pth')
model_3 = torch.load('models/BERTTransfer.pth')
model_4 = torch.load('models/RoBERTaTransfer.pth')
model_5 = torch.load('models/BERTTransferIrony.pth')
model_6 = torch.load('models/RoBERTaTransferIrony.pth')
model_7 = torch.load('models/BERTTransferHumor.pth')
model_8 = torch.load('models/RoBERTaTransferHumor.pth')

model.to(device)
roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_test_encodings = bert_tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)
roberta_test_encodings = roberta_tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)

bert_test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(bert_test_encodings['input_ids']),
    torch.tensor(bert_test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)


roberta_test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(roberta_test_encodings['input_ids']),
    torch.tensor(roberta_test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)

bert_test_loader = torch.utils.data.DataLoader(bert_test_dataset, batch_size=32, shuffle=True)
roberta_test_loader = torch.utils.data.DataLoader(roberta_test_dataset, batch_size=32, shuffle=True)

def calculate_metrics(model, loader):
    f1 = []
    precision = []
    recall = []
    
    f1_metric = BinaryF1Score().to(device)
    p_r = BinaryPrecisionRecallCurve().to(device)
    
    for idx, batch in enumerate(loader):
        # batch.to(device)
        outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        op = torch.argmax(outputs, axis = 1)
        
        f1.append(f1_metric(op, batch[2].to(device)).cpu().detach().numpy())
        # print(p_r(op.float(), batch[2].to(device)))
        # print(p)
        # print(r)
        
        # precision.append(p.cpu().detach().numpy())
        # recall.append(r.cpu().detach().numpy())
    
    return (sum(f1) / len(f1)) * 100


def print_metrics(model, loader, text):
    f1 = calculate_metrics(model, loader)
    print(f"F1 Score of {text} : {f1}")
    
    

print_metrics(model_1, bert_test_loader, "BERT: ")
print_metrics(model_2, roberta_test_loader, "RoBERTa: ")
print_metrics(model_3, bert_test_loader, "BERT Transfer IMDB: ")
print_metrics(model_4, roberta_test_loader, "RoBERTa Transfer IMDB: ")

print_metrics(model_5, bert_test_loader, "BERT Transfer Irony: ")
print_metrics(model_6, roberta_test_loader, "RoBERTa Transfer Irony: ")
print_metrics(model_7, bert_test_loader, "BERT Transfer Humor: ")
print_metrics(model_8, roberta_test_loader, "RoBERTa Transfer Humor: ")


F1 Score of BERT:  : 80.15630441467937
F1 Score of RoBERTa:  : 83.8939377447454
F1 Score of BERT Transfer IMDB:  : 79.46329567490554
F1 Score of RoBERTa Transfer IMDB:  : 82.4882546576058
F1 Score of BERT Transfer Irony:  : 77.55576052316805
F1 Score of RoBERTa Transfer Irony:  : 82.20451910321306
F1 Score of BERT Transfer Humor:  : 78.69429515629281
F1 Score of RoBERTa Transfer Humor:  : 82.35881212281018
